<a href="https://colab.research.google.com/github/macsakini/AI-Modelling-/blob/main/rlmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import gym
import keras
import tensorflow
#import keras_rl2
import random
import numpy as np

In [162]:
class BalanceEnv(gym.Env):
  #metadata = {‘render.modes’: [‘human’]}

  def __init__(self):
    #0 is going down, 1 is going up
    self.action_space = gym.spaces.Discrete(2,)
    #ranges from o to 7 metres
    self.observation_space = gym.spaces.Box(low = np.array([0]),high = np.array([7]))

    self.game_length = 20

    self.state = 3.5 + random.randint(-1,1)

    pass

  def step(self, action):
    if action == 0:
        self.state -= 1.25
        self.game_length -= 1
    else:
        self.state += 1.25
        self.game_length -= 1 
    
    if self.state >= 0 and self.state<= 7:
        reward = 1
    else:
        reward = -1

    if self.game_length <= 0:
        done = True
    elif self.state < 0:
        done = True
    elif self.state > 7:
        done =True
    else:
        done = False

    # Apply temperature noise
    #self.state += ra

    info = {}
    
    return self.state, reward, done, info

  def render(self):
    pass

  def reset(self):
    self.game_length = 50

    self.state = 3.5 + random.randint(-1,1)
    
    return self.state

In [167]:
env = BalanceEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [139]:
env.observation_space.sample()

array([2.6841073], dtype=float32)

In [159]:
episodes = 10
observation = env.reset()
for episode in range(1, episodes + 1):
    state = env.reset()
    
    done = False
    
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        #print(n_state)
        score += reward
    
    print('Episode:{} Score:{} Steps:{}'.format(episode, score, n_state))

Episode:1 Score:20 Steps:2.0
Episode:2 Score:1 Steps:8.25
Episode:3 Score:20 Steps:2.0
Episode:4 Score:1 Steps:-0.25
Episode:5 Score:7 Steps:-1.25
Episode:6 Score:2 Steps:7.5
Episode:7 Score:3 Steps:-0.25
Episode:8 Score:2 Steps:7.5
Episode:9 Score:11 Steps:8.25
Episode:10 Score:15 Steps:8.25


In [140]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [141]:
states = env.observation_space.shape
actions = env.action_space.n

In [136]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [150]:
model = build_model(states, actions)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 24)                48        
_________________________________________________________________
dense_18 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 50        
Total params: 698
Trainable params: 698
Non-trainable params: 0
_________________________________________________________________


In [112]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [149]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [151]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate = 1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=4000, visualize=False, verbose=1)

Training for 4000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    1/10000 [..............................] - ETA: 53:45 - reward: 1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 3995/10000 [==========>...................] - ETA: 57s - reward: 0.9840done, took 38.333 seconds


In [177]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))
print(scores)

Testing for 10 episodes ...
Episode 1: reward: 50.000, steps: 50
Episode 2: reward: 50.000, steps: 50
Episode 3: reward: 50.000, steps: 50
Episode 4: reward: 50.000, steps: 50
Episode 5: reward: 50.000, steps: 50
Episode 6: reward: 50.000, steps: 50
Episode 7: reward: 50.000, steps: 50
Episode 8: reward: 50.000, steps: 50
Episode 9: reward: 50.000, steps: 50
Episode 10: reward: 50.000, steps: 50
50.0


In [178]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)